In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
URI = "neo4j://localhost:7999"
AUTH = ("neo4j", "password")

In [4]:
dataset = pd.read_csv("../datasets/BestBooksEverClean.csv")
dataset

,title,series,author,genres,pages,publishDate,publishYear,rating,likedPercent,price
0,Why Are You a Vegan? and Other Wacky Verse for...,standalone,Violet's Vegan Comics,['Poetry'],116,January 5th 2015,2015,5.00,100.0,81.15
1,"You, My Love: A Diary in Verse",standalone,Richard Atwood,['Poetry'],116,February 10th 2008,2008,5.00,100.0,8.99
2,CIRCUS RIDER,standalone,Peter Breschard,['Historical Fiction'],330,December 15th 2010,2010,5.00,100.0,19.59
3,Bertie's Book of Spooky Wonders,standalone,Ocelot Emerson,"['Middle Grade', 'Fantasy']",244,October 15th 2019,2019,5.00,100.0,14.19
4,Manhood Of Humanity - The Science And Art Of H...,standalone,Alfred Korkzybski,['Science'],280,April 4th 2010,2010,5.00,100.0,22.40
5,100 XIV,standalone,Jerry Seguin,['Art'],104,December 14th 2013,2013,5.00,100.0,23.55
6,Maze of Existence,Mystic Deja,Tina M. Randolph,['Fantasy'],178,October 10th 2002,2002,5.00,100.0,7.97
7,The Illustrated Vivian Stanshall: A Fairytale ...,standalone,Ki Longfellow,"['Art', 'Biography', 'Memoir']",326,February 5th 2018,2018,4.93,99.0,56.86
8,Orion: The Fight for Vox,Voxian,Ruth Watson-Morris,['Fantasy'],314,August 11th 2012,2012,4.93,100.0,13.74
9,A Debt Free You,standalone,Steve Julien,"['Finance', 'Nonfiction']",104,March 1st 2012,2012,4.92,100.0,38.83


#### From String list, return the list of genres

In [51]:
def genres_to_list(genres: str) -> list:
  list_of_genres = genres.removeprefix('[').removesuffix(']').split(', ')
  return list(map(lambda s: s[1:][:-1], list_of_genres))

In [52]:
dataset["genres"] = dataset["genres"].apply(genres_to_list)
type(dataset['genres'].iloc[0])

list

#### Enable Connection

In [53]:
driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session()

In [157]:
def insert_user(name: str) -> bool:
  # Insertar usuario
  result = session.run(f"MATCH (u:User) WHERE u.name = $name RETURN u.name as name", {"name": name})
  if len(result.data()) == 0:
    try:
      session.run("CREATE (u:User {name: $name})", {"name": name})
      return True
    except:
      pass
  return False

def show_users():
  # Mostrar usuarios
  print("Usuarios:")
  result = session.run("MATCH (u:User) RETURN u.name AS name")
  for record in result:
    print(f" - {record["name"]}")

def show_connections():
  # Mostrar vinculos
  print("\nVinculos:")
  result = session.run("MATCH (u:User)-[r:FRIEND]->(u2:User) RETURN u.name AS name, u2.name AS name2")
  for record in result:
    print(f" - {record['name']} es amigo de {record['name2']}")

def insert_author(author: str) -> bool:
  # Insertar autor
  result = session.run("MATCH (a:Author) WHERE a.name = $name RETURN a.name as name", {"name": author})
  if len(result.data()) == 0:
    try:
      session.run("CREATE (a:Author {name: $name})", {"name": author})
      return True
    except:
      pass
  return False

def show_authors():
  # Mostrar autores
  print("Autores:")
  result = session.run("MATCH (a:Author) RETURN a.name AS name")
  for record in result:
    print(f" - {record["name"]}")

def insert_genre(genre: str) -> bool:
  # Insertar Genero
  result = session.run("MATCH (g:Genre) WHERE g.name = $name RETURN g.name as name", {"name": genre})
  if len(result.data()) == 0:
    try:
      session.run("CREATE (g:Genre {name: $name})", {"name": genre})
      return True
    except:
      pass
  return False

def show_genres():
  # Mostrar generos
  print("Generos:")
  result = session.run("MATCH (g:Genre) RETURN g.name AS name")
  for record in result:
    print(f" - {record["name"]}")

def insert_books(books: pd.DataFrame) -> bool:
  # Insertar libros
  query = """
    UNWIND $books AS book
    MERGE (b:Book {title: book.title})
    ON CREATE SET b.author = book.author, 
                  b.pages = book.pages, b.rating = book.rating, b.likedPercent = book.likedPercent, 
                  b.numRating = book.numRating, b.price = book.price, b.publishDate = book.publishDate, 
                  b.publishYear = book.publishYear
    """
  
  try:
    books_dict = books.to_dict('records')
    session.run(query, {"books": books_dict})
    return True
  except:
    pass
  return False

def get_random_books(n: int):
  # Funcion para obtener n random books
  query = f"""
    MATCH (b:Book)
    RETURN b.title AS title, b.author AS author, b.pages AS pages,
           b.rating AS rating, b.likedPercent AS likedPercent, b.numRating AS numRating,
           b.price AS price, b.publishDate AS publishDate, b.publishYear AS publishYear
    ORDER BY rand()
    LIMIT {n}
    """
  result = session.run(query)
  books = result.data()
  return books

def friendship_relation(tx, user_a: str, user_b: str):
  # Funcion para enlazar usuarios como amigos
  query = (
      "MATCH (u1:User {name: $name_a}), (u2:User {name: $name_b}) "
      "CREATE (u1)-[:FRIEND]->(u2)"
  )
  tx.run(query, name_a=user_a, name_b=user_b)

def book_genre_relation(tx, book_title: str, genres: list[str]):
  # Funcion para enlazar libro con su genero
  for genre in genres:
    query = (
              "MATCH (b:Book {title: $book_title}), (g:Genre {name: $genre}) "
              "CREATE (b)-[:BELONGS_TO]->(g)"
          )
    tx.run(query, book_title=book_title, genre=genre)

def author_book_relation(tx, author_name: str, book_titles: list[str]):
  # Funcion para enlazar autor con su obra
  for book_title in book_titles:
    query = (
              "MATCH (b:Book {title: $book_title}), (a:Author {name: $name}) "
              "CREATE (a)-[:WROTE]->(b)"
          )
    tx.run(query, book_title=book_title, name=author_name)

def user_book_likes_relation(tx, user_name: str, book_title: str):
  # Funcion para enlazar libros con usuarios mediante un like
  query = (
    "MATCH (b:Book {title: $title}), (u:User {name: $name}) "
    "CREATE (u)-[:LIKES]->(b)"
  )
  tx.run(query, title=book_title, name=user_name)

def make_relation(relation, org: str, dst: str | list[str]):
  # Crear relacion dependiendo de la funcion que se le pase
  session.execute_write(relation, org, dst)

def show_friend_recomendations(user_name: str, limit=5):
  # Recomendar libros en base a los gustos de los amigos
  query = (
    "MATCH (u:User {name: $user_name})-[:FRIEND]->()-[:LIKES]->(b:Book) "
    "WHERE NOT (u)-[:LIKES]->(b) "
    "RETURN b.title AS recommended_book, COUNT(*) AS friend_likes "
    "ORDER BY friend_likes DESC "
    "LIMIT $limit;"
  )

  try:
    result = session.run(query, {"user_name": user_name, "limit": limit})
    df = pd.DataFrame([record.data() for record in result])
    df.columns = ["Book Title", "Friend Likes"]
    return df
  except:
    return None


#### Insert Users

In [55]:
users = [
  'Victor',
  'Badre',
  'Daniel',
  'Adrian',
  'Manuel',
  'Angel',
  'Felix',
  'Fernando'
]

In [56]:
for user in users:
  insert_user(user)

In [57]:
show_users()

Usuarios:
 - Victor
 - Badre
 - Daniel
 - Adrian
 - Manuel
 - Angel
 - Felix
 - Fernando


#### Insert authors

In [58]:
for author in dataset['author'].unique():
  insert_author(author)

In [59]:
show_authors()

Autores:
 - Violet's Vegan Comics
 - Richard Atwood
 - Peter Breschard
 - Ocelot Emerson
 - Alfred Korkzybski
 - Jerry Seguin
 - Tina M. Randolph
 - Ki Longfellow
 - Ruth Watson-Morris
 - Steve Julien
 - Ali Marsman
 - Hudith F. Dolkart
 - Brae Wyckoff
 - Jacob Lasher
 - Sita Bennett
 - Bernie Morris
 - Rohith S. Katbamna
 - Martha A. Cheves
 - Jo-Anne McArthur
 - Bill Watterson
 - Francis Shenstone
 - Aimee Lewis
 - J.D. Estrada
 - Brandon Sanderson
 - Victoria Botkin
 - William Shakespeare
 - Pythia Peay
 - Janet G. Travell
 - Rachna Khemchandani
 - J.K. Rowling
 - Banani Ray
 - Laurick Ingram
 - Anonymous
 - Francine Rivers
 - Lawrence Hole
 - George Herriman
 - G.B. Hope
 - George Mendoza
 - Marika Germanis
 - Neil Gaiman
 - Amit Ray
 - Fletcher McHale
 - S.F. Mazhar
 - Rich Okun
 - Andrew Peterson
 - Georg Trakl
 - Gary Collins
 - Nick Brandt
 - Rien Poortvliet
 - Sue Lloyd-Roberts
 - Karen Kingsbury
 - Sarah J. Maas
 - Michelle R. Eastman
 - Amanda Barratt
 - Teric Darken
 - Lisa

#### Insert Genres

In [60]:
set_of_genres = set()
for genres in dataset['genres'].values:
  for genre in genres:
    set_of_genres.add(genre)
set_of_genres

{'14th Century',
 '16th Century',
 '18th Century',
 '19th Century',
 '1st Grade',
 '20th Century',
 '21st Century',
 'Abuse',
 'Academic',
 'Action',
 'Activism',
 'Adult',
 'Adult Fiction',
 'Adventure',
 'Aeroplanes',
 'Africa',
 'African American',
 'African American Literature',
 'African American Romance',
 'Agriculture',
 'Alchemy',
 'Aliens',
 'Alternate History',
 'American',
 'American Civil War',
 'American History',
 'Americana',
 'Ancient',
 'Ancient History',
 'Angels',
 'Animal Fiction',
 'Animals',
 'Anime',
 'Anthologies',
 'Anthropology',
 'Anthropomorphic',
 'Anti Racist',
 'Architecture',
 'Art',
 'Art History',
 'Art and Photography',
 'Asia',
 'Asian Literature',
 'Astrology',
 'Astronomy',
 'Atheism',
 'Audiobook',
 'Australia',
 'Autobiography',
 'Aviation',
 'BDSM',
 'Baha I',
 'Batman',
 'Biblical',
 'Biblical Fiction',
 'Biography',
 'Biography Memoir',
 'Biology',
 'Birds',
 'Boarding School',
 'Book Club',
 'Books About Books',
 'British Literature',
 'Buddh

In [61]:
len(set_of_genres)

453

In [62]:
for genre in set_of_genres:
  insert_genre(genre)

In [63]:
show_genres()

Generos:
 - Linguistics
 - Latin American History
 - Nutrition
 - Business
 - Disability
 - Lesbian Romance
 - Sword and Sorcery
 - Childrens
 - European History
 - Murder Mystery
 - Dinosaurs
 - Audiobook
 - Christian Romance
 - Technology
 - Outdoors
 - Crafts
 - True Crime
 - Regency
 - 20th Century
 - New Testament
 - American Civil War
 - Field Guides
 - Historical
 - Adult Fiction
 - Kids
 - Software
 - Counselling
 - New Adult Romance
 - Maritime
 - Tarot
 - Monsters
 - Psychoanalysis
 - Storytime
 - Race
 - School
 - Geography
 - Aeroplanes
 - Science Fiction Fantasy
 - Portugal
 - Buddhism
 - Vegan
 - Americana
 - Spanish History
 - Comic Strips
 - Realistic Fiction
 - 18th Century
 - Diary
 - Police
 - Language
 - Superheroes
 - Horror
 - Asia
 - German Literature
 - India
 - Lds
 - War
 - High Fantasy
 - Soccer
 - Computer Science
 - World War I
 - Leadership
 - Space Opera
 - Colouring Books
 - Education
 - Picture Books
 - Family
 - Military Romance
 - Alchemy
 - Angels
 -

#### Insert books

In [64]:
insert_books(dataset)

True

In [65]:
books = get_random_books(5)
[f"{b["title"]} - {b["author"]}" for b in books]

['Efrén Divided - Ernesto Cisneros',
 'Wolves A Photographic Celebration - Amber Rose',
 'The Sun Does Shine: How I Found Life and Freedom on Death Row - Anthony Ray Hinton',
 'Locke & Key, Vol. 6: Alpha & Omega - Joe Hill',
 'Later Short Stories, 1888-1903 - Anton Chekhov']

#### Friend relationship

In [82]:
user1 = "Victor"
user2 = "Badre"
make_relation(friendship_relation, user1, user2)
make_relation(friendship_relation, user2, user1)

True

In [162]:
# Mostrar vinculos
print("\nAmistades:")
result = session.run("MATCH (u:User)-[r:FRIEND]->(u2:User) RETURN u.name AS name, u2.name AS name2")
for record in result:
  print(f" - {record['name']} es amigo de {record['name2']}")


Amistades:
 - Angel es amigo de Victor
 - Fernando es amigo de Victor
 - Badre es amigo de Victor
 - Daniel es amigo de Victor
 - Adrian es amigo de Badre
 - Victor es amigo de Badre
 - Victor es amigo de Daniel
 - Badre es amigo de Adrian
 - Felix es amigo de Adrian
 - Victor es amigo de Angel
 - Adrian es amigo de Felix
 - Fernando es amigo de Felix
 - Felix es amigo de Fernando
 - Victor es amigo de Fernando


#### Genre relationship

In [68]:
for _, row in dataset.iterrows():
  book_title, genres = row["title"], row["genres"]
  make_relation(book_genre_relation, book_title, genres)

In [69]:
print("\nLibros relacionados con su genero:")
result = session.run("MATCH (b:Book)-[r:BELONGS_TO]->(g:Genre) RETURN b.title AS title, g.name AS name")
for record in result:
  print(f" - {record['title']} pertenece a {record['name']}")


Vinculos:
 - The Oxford English Dictionary (20 Volume Set) pertenece a Linguistics
 - Century of the Wind pertenece a Latin American History
 - How Not to Die: Discover the Foods Scientifically Proven to Prevent and Reverse Disease pertenece a Nutrition
 - Shoe Dog: A Memoir by the Creator of NIKE pertenece a Business
 - A Newborn Business: Esports pertenece a Business
 - Freedom from Bosses Forever pertenece a Business
 - Poor Charlie's Almanack: The Wit and Wisdom of Charles T. Munger pertenece a Business
 - Dave Ramsey's Financial Peace University pertenece a Business
 - Berkshire Hathaway Letters to Shareholders pertenece a Business
 - Multilingual Digital Marketing: How To Achieve Your Digital Marketing Objectives And Increase Sales pertenece a Business
 - Before and After the Book Deal: A Writer’s Guide to Finishing, Publishing, Promoting, and Surviving Your First Book pertenece a Business
 - Skallagrigg pertenece a Disability
 - Thank You, Mr. Falker pertenece a Disability
 - T

#### Author with book relation

In [70]:
for author in dataset["author"].unique():
  list_of_books = list(dataset[dataset["author"] == author]["title"].values)
  make_relation(author_book_relation, author, list_of_books)

In [161]:
print("\nLibros relacionados con autor:")
result = session.run("MATCH (a:Author)-[r:WROTE]->(b:Book) RETURN b.title AS title, a.name AS name")
for record in result:
  print(f" - {record['title']} pertenece a {record['name']}")


Libros relacionados con autor:
 - Why Are You a Vegan? and Other Wacky Verse for Kids pertenece a Violet's Vegan Comics
 - You, My Love: A Diary in Verse pertenece a Richard Atwood
 - CIRCUS RIDER pertenece a Peter Breschard
 - Bertie's Book of Spooky Wonders pertenece a Ocelot Emerson
 - Manhood Of Humanity - The Science And Art Of Human Engineering pertenece a Alfred Korkzybski
 - 100 XIV pertenece a Jerry Seguin
 - Maze of Existence pertenece a Tina M. Randolph
 - The Illustrated Vivian Stanshall: A Fairytale of Grimm Art pertenece a Ki Longfellow
 - Orion: The Fight for Vox pertenece a Ruth Watson-Morris
 - A Debt Free You pertenece a Steve Julien
 - Heal With Me & Our Journey Goes On pertenece a Ali Marsman
 - Book Too: Because of Grandmother pertenece a Ali Marsman
 - James Tissot: The Life of Christ pertenece a Hudith F. Dolkart
 - The Dragon God pertenece a Brae Wyckoff
 - The Vampire King (The Horn King, #3) pertenece a Brae Wyckoff
 - More To Life pertenece a Jacob Lasher
 -

In [145]:
dataset[dataset["genres"].apply(lambda x: "Science" in x)]

,title,series,author,genres,pages,publishDate,publishYear,rating,likedPercent,price
4,Manhood Of Humanity - The Science And Art Of H...,standalone,Alfred Korkzybski,[Science],280,April 4th 2010,2010,5.00,100.0,22.40
130,The Sibley Guide to Birds,standalone,David Allen Sibley,"[Reference, Birds, Nature, Nonfiction, Science...",544,October 3rd 2000,2000,4.67,99.0,7.35
203,"Edible Forest Gardens, Volume 2: Ecological De...",standalone,Dave Jacke,"[Gardening, Nonfiction, Sustainability, Agricu...",672,October 20th 2005,2005,4.63,100.0,43.23
257,The Feynman Lectures on Physics,standalone,Richard P. Feynman,"[Science, Physics, Nonfiction, Textbooks, Refe...",1552,August 8th 2005,2005,4.60,98.0,169.88
337,"Dinosaurs: The Most Complete, Up-to-Date Encyc...",standalone,Thomas R. Holtz Jr.,"[Dinosaurs, Science, Nonfiction, Palaeontology...",432,October 23rd 2007,2007,4.57,98.0,19.16
378,The Boy Who Was Raised as a Dog: And Other Sto...,standalone,Bruce D. Perry,"[Psychology, Nonfiction, Mental Health, Scienc...",288,January 8th 2007,2007,4.56,99.0,14.51
385,"Pharmako/Poeia: Plant Powers, Poisons, and Her...",standalone,Dale Pendell,"[Nonfiction, Plants, Nature, Herbs, Poetry, Sc...",304,February 1st 1994,1994,4.56,98.0,8.84
413,"Braiding Sweetgrass: Indigenous Wisdom, Scient...",standalone,Robin Wall Kimmerer,"[Nonfiction, Nature, Science, Environment, Mem...",391,October 15th 2013,2013,4.56,97.0,52.26
419,The Complete Works of Aristotle: The Revised O...,The Complete Works of Aristotle,Aristotle,"[Philosophy, Classics, Nonfiction, Science, Re...",1644,September 21st 1984,1984,4.55,98.0,29.95
443,Planet Earth II,standalone,Stephen Moss,"[Nonfiction, Nature, Science, Environment]",312,October 6th 2016,2016,4.55,100.0,1.81


#### Likes

In [150]:
make_relation(user_book_likes_relation, "Victor", "The Way of Kings")
make_relation(user_book_likes_relation, "Badre", "Harry Potter and the Prisoner of Azkaban")
make_relation(user_book_likes_relation, "Daniel", "Harry Potter and the Prisoner of Azkaban")
make_relation(user_book_likes_relation, "Felix", "Planet Earth II")
make_relation(user_book_likes_relation, "Angel", "A Man on the Moon")
make_relation(user_book_likes_relation, "Fernando", "Harry Potter and the Prisoner of Azkaban")
make_relation(user_book_likes_relation, "Adrian", "Where it all Began")

In [160]:
print("\nLikes:")
result = session.run("MATCH (u:User)-[r:LIKES]->(b:Book) RETURN b.title AS title, u.name AS name")
for record in result:
  print(f" - A {record['name']} le gusta {record['title']}")


Likes:
 - A Victor le gusta The Way of Kings
 - A Badre le gusta Harry Potter and the Prisoner of Azkaban
 - A Daniel le gusta Harry Potter and the Prisoner of Azkaban
 - A Angel le gusta A Man on the Moon
 - A Fernando le gusta Harry Potter and the Prisoner of Azkaban
 - A Adrian le gusta Where it all Began
 - A Felix le gusta Planet Earth II


#### Book Recomndations By Likes

In [159]:
show_friend_recomendations('Badre')

,Book Title,Friend Likes
0,The Way of Kings,1
1,Where it all Began,1


#### Close Connection

In [73]:
# session.close()
# driver.close()